In [1]:
import json
import pickle
import pandas as pd

In [2]:
with open(r'../data/geojson/gadm36_IND_1.json','r') as f:
    districts=json.load(f)

In [3]:
for i in districts['features']:
    i['id']=i['properties']['NAME_1'].lower()

In [4]:
with open(r'../data/geojson/gadm36_IND_1_id.json','w') as f:
    json.dump(districts,f)

In [22]:
with open(r'../data/groundwater-district-monthly.pickle','rb') as f:
    df=pickle.load(f)
df['district']=df.index

In [30]:
found=[]
for i in districts['features']:
    if i['id'] in list(df.index):
        found.append(i['id'])

KeyError: 0

In [28]:
len(found)

593